<a href="https://colab.research.google.com/github/alex-tikh/counting-pigs/blob/dev/prediction_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda2-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py27_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py27_0
    - cffi==1.14.0=py27he30daa8_1
    - chardet==3.0.4=py27_1003
    - conda-package-handling==1.6.0=py27h7b6447c_0
    - conda==4.8.3=py27_0
    - cryptography==2.8=py27h1ba5d50_0
    - enum34==1.1.6=py27_1
    - futures==3.3.0=py27_0
    - idna==2.8=py27_0
    - ipaddress==1.0.23=py_0
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.3=he6710b0_1
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1g=h7b6447c_0
    - pip==19.3.1=py27_0
    - pycosat==0.6.3=py27h7b6447c_0
    - pycparser==2.20=py_0
    - pyopenssl==19.1.0=py27_0
    - pysocks==1.7.1=py27_0
    - python==2.7.18=h15b4118_1
    - readline==8.0=h7b6447c_0
    - requ

--2021-02-25 06:47:47--  https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda2-latest-Linux-x86_64.sh [following]
--2021-02-25 06:47:47--  https://repo.anaconda.com/miniconda/Miniconda2-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51093759 (49M) [application/x-sh]
Saving to: ‘Miniconda2-latest-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 9.24M 5s
    50K .......... .......... .......... .......... ..........

In [3]:
!python --version

Python 2.7.18 :: Anaconda, Inc.


In [4]:
!conda env create -f environment.yml

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



h5py-2.8.0           | 896 KB    | : 100% 1.0/1 [00:00<00:00,  5.43s/it]           
boost-1.66.0         | 319 KB    | : 100% 1.0/1 [00:00<00:00,  2.78s/it]            
bzip2-1.0.8          | 78 KB     | : 100% 1.0/1 [00:00<00:00,  6.40it/s]
attrs-20.3.0         | 41 KB     | : 100% 1.0/1 [00:00<00:00, 15.28it/s]
subprocess32-3.5.4   | 51 KB     | : 100%

In [ ]:
%%bash
source activate dymov

python
import sys
# some simple python commands
sys.path.append('/content/drive/MyDrive/counting-pigs/src')
sys.path.append('/usr/local/lib/python2.7/site-packages')

#import sys
#sys.path.append('~/caffe/python/')
#sys.path.append('~/caffe/python/caffe')
#sys.path.insert(0,'~/caffe/python/')
import h5py
import scipy.io as sio
import matplotlib.pyplot as plt

# System
import signal
import sys, getopt
signal.signal(signal.SIGINT, signal.SIG_DFL)
import time

# Vision and maths
import numpy as np
from matplotlib import cm as CM
import utils as utl
from gen_features import genDensity, genPDensity, loadImage, extractEscales
# import caffe_header
import caffe
import cv2

class CaffePredictor:
    def __init__(self, prototxt, caffemodel, n_scales):       
        # Load a precomputed caffe model
        self.net = caffe.Net(prototxt, caffemodel, caffe.TEST)
        
        # input preprocessing: 'data' is the name of the input blob == net.inputs[0]
        self.transformer = caffe.io.Transformer({'data': self.net.blobs['data_s0'].data.shape})
        self.transformer.set_transpose('data', (2, 0, 1)) # It's already RGB
        # Reshape net for the single input
        b_shape = self.net.blobs['data_s0'].data.shape
        self._n_scales = n_scales
        for s in range(n_scales):
            scale_name = 'data_s{}'.format(s)
            self.net.blobs[scale_name].reshape(b_shape[0],b_shape[1],b_shape[2],b_shape[3])

    # Probably it is not the eficient way to do it...
    def process(self, im, base_pw):
        # Compute dense positions where to extract patches
        [heith, width] = im.shape[0:2]
        pos = utl.get_dense_pos(heith, width, base_pw, stride=3)

        # Initialize density matrix and vouting count
        dens_map = np.zeros( (heith, width), dtype = np.float32 )   # Init density to 0
        count_map = np.zeros( (heith, width), dtype = np.int32 )     # Number of votes to divide

        # print('pos',len(pos))
        # exit(0)
        
        # Iterate for all patches
        for ix, p in enumerate(pos):
            # Compute displacement from centers
            dx=dy=int(base_pw/2)
    
            # Get roi
            x,y=p
            sx=slice(x-dx,x+dx+1,None)
            sy=slice(y-dy,y+dy+1,None)
            crop_im=im[sx,sy,...]
            h, w = crop_im.shape[0:2]
            if h!=w or (h<=0):
                continue
            
            # Get all the scaled images
            im_scales = extractEscales([crop_im], self._n_scales)
            
            # Load and forward CNN
            for s in range(self._n_scales):
                data_name = 'data_s{}'.format(s)
                self.net.blobs[data_name].data[...] = self.transformer.preprocess('data', im_scales[0][s].copy())
            self.net.forward()
            
            # Take the output from the last layer
            # Access to the last layer of the net, second element of the tuple (layer, caffe obj)
            pred = self.net.blobs.items()[-1][1].data
            
            # Make it squared
            p_side = int(np.sqrt( len( pred.flatten() ) )) 
            pred = pred.reshape(  (p_side, p_side) )
            
            # Resize it back to the original size
            pred = utl.resizeDensityPatch(pred, crop_im.shape[0:2])          
            pred[pred<0] = 0

            pred = cv2.blur(pred,(9,9))

            # Sumup density map into density map and increase count of votes
            dens_map[sx,sy] += pred
            count_map[sx,sy] += 1

            # kernel = np.ones((1,1),dtype='float')*(1.0/(1*1))
            # dens_map = cv2.filter2D(dens_map,-1,kernel)

        # Remove Zeros
        count_map[ count_map == 0 ] = 1



        # kernel = np.ones((1,1),dtype='float')*(1.0/(1*1))
        # dens_map = cv2.filter2D(dens_map,-1,kernel)

        # Average density map
        dens_map = dens_map / count_map        
        
        return dens_map

def initTestFromCfg(cfg_file):
    '''
    @brief: initialize all parameter from the cfg file. 
    '''
    
    # Load cfg parameter from yaml file
    cfg = utl.cfgFromFile(cfg_file)
    
    # Fist load the dataset name
    dataset = cfg.DATASET
    
    # Set default values
    use_mask = cfg[dataset].USE_MASK
    use_perspective = cfg[dataset].USE_PERSPECTIVE
    
    # Mask pattern ending
    mask_file = cfg[dataset].MASK_FILE
        
    # Img patterns ending
    dot_ending = cfg[dataset].DOT_ENDING
    
    # Test vars
    test_names_file = cfg[dataset].TEST_LIST
    
    # Im folder
    im_folder = cfg[dataset].IM_FOLDER
    
    # Results output foder
    results_file = cfg[dataset].RESULTS_OUTPUT

    # Resize image
    resize_im = cfg[dataset].RESIZE

    # Patch parameters
    pw = cfg[dataset].PW # Patch with 
    sigmadots = cfg[dataset].SIG # Densities sigma
    n_scales = cfg[dataset].N_SCALES # Escales to extract
    perspective_path = cfg[dataset].PERSPECTIVE_MAP
    is_colored = cfg[dataset].COLOR

        
    return (dataset, use_mask, mask_file, test_names_file, im_folder, 
            dot_ending, pw, sigmadots, n_scales, perspective_path, 
            use_perspective, is_colored, results_file, resize_im)

# Init parameters      
use_cpu = True
gpu_dev = 0

# GAME max level
mx_game = 4 # Max game target

# Batch size
b_size = -1

# CNN vars
# prototxt_path = 'models/trancos/hydra2/hydra_deploy.prototxt'
# caffemodel_path = 'models/trancos/hydra2/trancos_hydra2.caffemodel'

prototxt_path='/content/drive/MyDrive/counting-pigs/models/trancos/ccnn/ccnn_deploy.prototxt'
caffemodel_path='/content/drive/MyDrive/counting-pigs/our_scale/ccnn_trancos_iter.caffemodel'
cfg_file='/content/drive/MyDrive/counting-pigs/models/trancos/ccnn/ccnn_trancos_cfg.yml'

(dataset, use_mask, mask_file, test_names_file, im_folder, 
        dot_ending, pw, sigmadots, n_scales, perspective_path, 
        use_perspective, is_colored, results_file, resize_im) = initTestFromCfg(cfg_file)

# Set GPU CPU setting
if use_cpu:
    caffe.set_mode_cpu()
else:
    # Use GPU
    caffe.set_device(gpu_dev)
    caffe.set_mode_gpu()

if use_perspective:
    pers_file = h5py.File(perspective_path,'r')
    pmap = np.array( pers_file['pmap'] )
    pers_file.close()

mask = None
if dataset == 'UCSD':
    print "Reading mask"
    if use_mask:
        mask_f = h5py.File(mask_file,'r')
        mask = np.array(mask_f['mask'])
        mask_f.close()

print "Reading image file names:"
#im_names = np.loadtxt('.'+test_names_file, dtype='str')

# Init CNN
CNN = CaffePredictor(prototxt_path, caffemodel_path, n_scales)

#%%time
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50]
video_dir = '/content/drive/MyDrive/counting-pigs/dymov_ug/3.mp4'
vidcap = cv2.VideoCapture(video_dir)
success, image = vidcap.read()
count = 0
while success:
    if count % 6 == 0:
        print 'proccessing frame %d' % count
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        _, image = cv2.imencode('.jpg', image, encode_param)
        image = cv2.imdecode(image, 1)
        image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        resImg = CNN.process(image, pw) 
        heatmap = cv2.applyColorMap(
            np.uint8(255*(resImg - resImg.min())/(resImg.max() - resImg.min())),
            cv2.COLORMAP_JET
        ) 
        res = np.uint8(0.7*np.uint8(255*image)) + np.uint8(0.3*heatmap)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(res, str(resImg.sum().round()), (10, 450), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imwrite(video_dir[:-6]+"/3_%d.jpg" % count, res)
    count += 1
    success, image = vidcap.read()